In [1]:
!pip install -U transformers datasets accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling, pipeline
from datasets import load_dataset, Dataset
import torch
import json
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import sqlite3
import pandas as pd

db_path = "/content/drive/My Drive/chat_backup.db"

conn = sqlite3.connect(db_path)

first_chat = 'FIRST NUMBER'

first_query = f"""
SELECT
  datetime(m.date / 1000000000 + strftime('%s', '2001-01-01'), 'unixepoch') AS timestamp,
  m.is_from_me,
  COALESCE(h.id, 'You') AS sender,
  m.text
FROM
  message m
LEFT JOIN
  handle h ON m.handle_id = h.rowid
JOIN
  chat_message_join cmj ON m.rowid = cmj.message_id
JOIN
  chat c ON cmj.chat_id = c.rowid
WHERE
  c.chat_identifier = '{first_chat}'
  AND m.text IS NOT NULL
ORDER BY
  timestamp ASC;
"""


df1 = pd.read_sql_query(first_query, conn)

second_chat = 'SECOND NUMBER'

second_query = f"""
SELECT
  datetime(m.date / 1000000000 + strftime('%s', '2001-01-01'), 'unixepoch') AS timestamp,
  m.is_from_me,
  COALESCE(h.id, 'You') AS sender,
  m.text
FROM
  message m
LEFT JOIN
  handle h ON m.handle_id = h.rowid
JOIN
  chat_message_join cmj ON m.rowid = cmj.message_id
JOIN
  chat c ON cmj.chat_id = c.rowid
WHERE
  c.chat_identifier = '{second_chat}'
  AND m.text IS NOT NULL
ORDER BY
  timestamp ASC;
"""

df2 = pd.read_sql_query(second_query, conn)

third_chat = 'THIRD NUMBER'

third_query = f"""
SELECT
  datetime(m.date / 1000000000 + strftime('%s', '2001-01-01'), 'unixepoch') AS timestamp,
  m.is_from_me,
  COALESCE(h.id, 'You') AS sender,
  m.text
FROM
  message m
LEFT JOIN
  handle h ON m.handle_id = h.rowid
JOIN
  chat_message_join cmj ON m.rowid = cmj.message_id
JOIN
  chat c ON cmj.chat_id = c.rowid
WHERE
  c.chat_identifier = '{third_chat}'
  AND m.text IS NOT NULL
ORDER BY
  timestamp ASC;
"""

df3 = pd.read_sql_query(third_query, conn)

fourth_chat = 'FOURTH NUMBER'

fourth_query = f"""
SELECT
  datetime(m.date / 1000000000 + strftime('%s', '2001-01-01'), 'unixepoch') AS timestamp,
  m.is_from_me,
  COALESCE(h.id, 'You') AS sender,
  m.text
FROM
  message m
LEFT JOIN
  handle h ON m.handle_id = h.rowid
JOIN
  chat_message_join cmj ON m.rowid = cmj.message_id
JOIN
  chat c ON cmj.chat_id = c.rowid
WHERE
  c.chat_identifier = '{fourth_chat}'
  AND m.text IS NOT NULL
ORDER BY
  timestamp ASC;
"""

df4 = pd.read_sql_query(fourth_query, conn)

fifth_chat = 'FIFTH NUMBER'

fifth_query = f"""
SELECT
  datetime(m.date / 1000000000 + strftime('%s', '2001-01-01'), 'unixepoch') AS timestamp,
  m.is_from_me,
  COALESCE(h.id, 'You') AS sender,
  m.text
FROM
  message m
LEFT JOIN
  handle h ON m.handle_id = h.rowid
JOIN
  chat_message_join cmj ON m.rowid = cmj.message_id
JOIN
  chat c ON cmj.chat_id = c.rowid
WHERE
  c.chat_identifier = '{fifth_chat}'
  AND m.text IS NOT NULL
ORDER BY
  timestamp ASC;
"""

df5 = pd.read_sql_query(fifth_query, conn)

In [5]:
def parser(df):
    messages = df.to_dict("records")
    conversations = []
    i = 0

    while i < len(messages) - 1:
        msg = messages[i]

        # Only start a pair when the message is not me
        if msg["is_from_me"] == 0:
            incoming = [msg["text"]]
            i += 1

            # Collect multiple messages (before i reply)
            while i < len(messages) and messages[i]["is_from_me"] == 0:
                incoming.append(messages[i]["text"])
                i += 1

            # Now collect my reply/replies
            outgoing = []
            while i < len(messages) and messages[i]["is_from_me"] == 1:
                outgoing.append(messages[i]["text"])
                if i + 1 < len(messages) and messages[i + 1]["is_from_me"] == 0:
                    break
                i += 1

            # Save the pair if both sides have content
            if incoming and outgoing:
                prompt = "Friend: " + "\n".join(incoming) + "\nYou:"
                completion = " " + "\n".join(outgoing)
                conversations.append({
                    "prompt": prompt.strip(),
                    "completion": completion.strip()
                })
        else:
            i += 1
    return conversations

In [6]:
convo1 = parser(df1)
convo2 = parser(df2)
convo3 = parser(df3)
convo4 = parser(df4)
convo5 = parser(df5)
all_convos = convo1 + convo2 + convo3 + convo4 + convo5

In [7]:
dataset = Dataset.from_list(all_convos)

In [8]:
# Now split into train/test
split_dataset = dataset.train_test_split(test_size=0.1)

In [9]:
#tokenizing data
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
  text = example["prompt"] + example["completion"]
  return tokenizer(text, padding=True, truncation=True, max_length=1024)

tokenized = split_dataset.map(tokenize, batched=False)

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1964 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Flatten and ignore -100s
    loss = torch.nn.functional.cross_entropy(
        torch.tensor(logits).view(-1, logits.shape[-1]),
        torch.tensor(labels).view(-1),
        ignore_index=-100,
        reduction='mean'
    )
    return {"perplexity": math.exp(loss.item())}

In [17]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir = "./gpt2-finetuned",
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 1,
    num_train_epochs = 3,
    logging_steps = 100,
    save_total_limit = 2,
    fp16 = True,
    push_to_hub = False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized["train"],
    eval_dataset = tokenized["test"],
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
100,3.901200
200,3.497400
300,3.372200
400,3.256900
500,3.108800
600,3.283900
700,3.085200
800,3.215700
900,3.172300
1000,2.842400


TrainOutput(global_step=2946, training_loss=2.7703974582027273, metrics={'train_runtime': 409.149, 'train_samples_per_second': 14.401, 'train_steps_per_second': 7.2, 'total_flos': 126595989504000.0, 'train_loss': 2.7703974582027273, 'epoch': 3.0})

In [19]:
model.save_pretrained("Mymic")
tokenizer.save_pretrained("Mymic")

('Mymic/tokenizer_config.json',
 'Mymic/special_tokens_map.json',
 'Mymic/vocab.json',
 'Mymic/merges.txt',
 'Mymic/added_tokens.json',
 'Mymic/tokenizer.json')

In [21]:
generator = pipeline("text-generation", model = model, tokenizer = tokenizer)
prompt = "Friend:\nyou wanna hang?\nYou:"
output = generator(prompt, max_length = 50, do_sample = True, temperature = 0.6, top_p = 0.9)

print(output[0]["generated_text"])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Friend:
you wanna hang?
You:Yeah I’m down I’m going to Broadway 
You wanna get a table at Broadway 
I’m gonna be down at 7:30
You wanna come over?
I’m down I’m down 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
You wanna come over?
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I’m down Broadway 
I have a table at Broadway 
I’m down Broadway 
I have a table at Broadway 
I have a table at Broadway 
I have a table at Broadway 
I have a table at Broadway 
I have a table at Broadway 
You wanna come over


In [22]:
import re

def clean_two_line_response(generated_text):
    # Extract everything after "You:"
    match = re.search(r"You:\s*(.*)", generated_text, re.DOTALL)
    response = match.group(1) if match else generated_text

    # Split into non-empty lines
    lines = [line.strip() for line in response.strip().splitlines() if line.strip()]

    # Return the first two lines (joined by newline)
    return "\n".join(lines[:2])

In [23]:
prompt = "Friend:\nwanna hang?\nYou:"

raw_output = generator(prompt, max_length=60, do_sample=True, temperature=0.8)[0]["generated_text"]
cleaned = clean_two_line_response(raw_output)

print("Mymic:", cleaned)

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Mymic: I’m at Broadway at 5
You wanna hang this weekend?


In [24]:
import shutil
shutil.make_archive("Mymic", 'zip', "/content/Mymic")

'/content/Mymic.zip'

In [25]:
from google.colab import files
files.download("Mymic.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>